# リサイズとクロップ

このノートブックでは、リサイズとクロップについて学びます。

## 目次
1. [リサイズ](#リサイズ)
2. [クロップ](#クロップ)
3. [アスペクト比の保持](#アスペクト比の保持)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import filters, feature

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False
np.random.seed(42)
print("ライブラリのインポートが完了しました。")

## リサイズ

In [ ]:
# 画像のリサイズ
# 様々な方法で画像のサイズを変更します

# サンプル画像の作成
img = np.random.randint(0, 256, (200, 300, 3), dtype=np.uint8)

# OpenCVでのリサイズ
# 方法1: 指定サイズにリサイズ
resized_fixed = cv2.resize(img, (150, 100))

# 方法2: スケールファクターでリサイズ
resized_scale = cv2.resize(img, None, fx=0.5, fy=0.5)

# 方法3: 異なる補間手法
resized_nearest = cv2.resize(img, (150, 100), interpolation=cv2.INTER_NEAREST)
resized_linear = cv2.resize(img, (150, 100), interpolation=cv2.INTER_LINEAR)
resized_cubic = cv2.resize(img, (150, 100), interpolation=cv2.INTER_CUBIC)

# PILでのリサイズ
img_pil = Image.fromarray(img)
resized_pil = img_pil.resize((150, 100), Image.LANCZOS)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img)
axes[0, 0].set_title(f'元の画像 ({img.shape[1]}x{img.shape[0]})')
axes[0, 0].axis('off')

axes[0, 1].imshow(resized_fixed)
axes[0, 1].set_title(f'固定サイズ (150x100)')
axes[0, 1].axis('off')

axes[0, 2].imshow(resized_scale)
axes[0, 2].set_title(f'スケール 0.5x ({resized_scale.shape[1]}x{resized_scale.shape[0]})')
axes[0, 2].axis('off')

axes[1, 0].imshow(resized_nearest)
axes[1, 0].set_title('最近傍補間')
axes[1, 0].axis('off')

axes[1, 1].imshow(resized_linear)
axes[1, 1].set_title('双線形補間（推奨）')
axes[1, 1].axis('off')

axes[1, 2].imshow(resized_cubic)
axes[1, 2].set_title('双三次補間')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("補間手法の比較:")
print("- INTER_NEAREST: 最近傍、高速だが品質が低い")
print("- INTER_LINEAR: 双線形、バランスが良い（推奨）")
print("- INTER_CUBIC: 双三次、高品質だが遅い")

## クロップ

In [ ]:
# 画像のクロップ（切り出し）
# 画像の特定領域を切り出します

# サンプル画像の作成
img = np.random.randint(0, 256, (200, 300, 3), dtype=np.uint8)

# NumPyスライシングでクロップ
# 形式: img[y1:y2, x1:x2]
cropped_region = img[50:150, 100:200]

# 中央部分をクロップ
h, w = img.shape[:2]
center_crop = img[h//4:3*h//4, w//4:3*w//4]

# 複数の領域をクロップ
crop1 = img[0:100, 0:150]
crop2 = img[100:200, 150:300]

# PILでのクロップ
img_pil = Image.fromarray(img)
cropped_pil = img_pil.crop((100, 50, 200, 150))  # (left, top, right, bottom)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img)
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(cropped_region)
axes[0, 1].set_title('領域の切り出し (50:150, 100:200)')
axes[0, 1].axis('off')

axes[0, 2].imshow(center_crop)
axes[0, 2].set_title('中央部分の切り出し')
axes[0, 2].axis('off')

axes[1, 0].imshow(crop1)
axes[1, 0].set_title('領域1 (左上)')
axes[1, 0].axis('off')

axes[1, 1].imshow(crop2)
axes[1, 1].set_title('領域2 (右下)')
axes[1, 1].axis('off')

axes[1, 2].imshow(np.array(cropped_pil))
axes[1, 2].set_title('PILでのクロップ')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print(f"元の画像サイズ: {img.shape}")
print(f"切り出し後のサイズ: {cropped_region.shape}")
print(f"中央部分のサイズ: {center_crop.shape}")

## アスペクト比の保持

In [ ]:
# アスペクト比を保持したリサイズ
# 画像の縦横比を維持しながらリサイズします

# サンプル画像（縦長）
img = np.random.randint(0, 256, (300, 200, 3), dtype=np.uint8)

# アスペクト比を保持したリサイズ
def resize_with_aspect_ratio(img, max_width=None, max_height=None):
    """アスペクト比を保持してリサイズ"""
    h, w = img.shape[:2]
    
    if max_width and max_height:
        # 両方指定された場合、小さい方に合わせる
        scale_w = max_width / w
        scale_h = max_height / h
        scale = min(scale_w, scale_h)
    elif max_width:
        scale = max_width / w
    elif max_height:
        scale = max_height / h
    else:
        return img
    
    new_w = int(w * scale)
    new_h = int(h * scale)
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

# 異なるサイズにリサイズ
resized_1 = resize_with_aspect_ratio(img, max_width=150)
resized_2 = resize_with_aspect_ratio(img, max_height=100)
resized_3 = resize_with_aspect_ratio(img, max_width=200, max_height=150)

# PILのthumbnailメソッド（アスペクト比保持）
img_pil = Image.fromarray(img)
img_pil.thumbnail((150, 100), Image.LANCZOS)
resized_pil = np.array(img_pil)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img)
axes[0, 0].set_title(f'元の画像 ({img.shape[1]}x{img.shape[0]}, アスペクト比: {img.shape[1]/img.shape[0]:.2f})')
axes[0, 0].axis('off')

axes[0, 1].imshow(resized_1)
axes[0, 1].set_title(f'最大幅150 ({resized_1.shape[1]}x{resized_1.shape[0]}, アスペクト比: {resized_1.shape[1]/resized_1.shape[0]:.2f})')
axes[0, 1].axis('off')

axes[0, 2].imshow(resized_2)
axes[0, 2].set_title(f'最大高さ100 ({resized_2.shape[1]}x{resized_2.shape[0]}, アスペクト比: {resized_2.shape[1]/resized_2.shape[0]:.2f})')
axes[0, 2].axis('off')

axes[1, 0].imshow(resized_3)
axes[1, 0].set_title(f'最大200x150 ({resized_3.shape[1]}x{resized_3.shape[0]}, アスペクト比: {resized_3.shape[1]/resized_3.shape[0]:.2f})')
axes[1, 0].axis('off')

axes[1, 1].imshow(resized_pil)
axes[1, 1].set_title(f'PIL thumbnail ({resized_pil.shape[1]}x{resized_pil.shape[0]}, アスペクト比: {resized_pil.shape[1]/resized_pil.shape[0]:.2f})')
axes[1, 1].axis('off')

# アスペクト比を保持しない場合との比較
resized_no_aspect = cv2.resize(img, (150, 100))
axes[1, 2].imshow(resized_no_aspect)
axes[1, 2].set_title(f'アスペクト比無視 ({resized_no_aspect.shape[1]}x{resized_no_aspect.shape[0]}, アスペクト比: {resized_no_aspect.shape[1]/resized_no_aspect.shape[0]:.2f})')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nアスペクト比の保持:")
print("元の画像のアスペクト比を維持することで、画像の歪みを防ぎます。")
print("機械学習の前処理などで重要です。")